# New Product Listing Checker and Twilio SMS Alert

In [ ]:
import datetime
import os
import subprocess
import time

import requests
from bs4 import BeautifulSoup
from IPython.display import clear_output

In [ ]:
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
    "dnt": "1",
}

In [ ]:
def get_and_parse_urls(
    urls: list[str], headers: dict[str, str], sleep_seconds: int = 1
) -> dict:
    """
    Gets website content and parses for product listings
    """
    data = {}
    for each_url in urls:
        time.sleep(sleep_seconds)
        # get content
        each_response = requests.get(each_url, headers=headers)
        # parse content
        each_content = BeautifulSoup(each_response.content)
        each_divs = each_content.find_all("div", {"class": "desc"})
        for each_div in each_divs:
            each_div_str = str(each_div)
            if "price" in each_div_str:
                each_link_start = each_div_str.find('<a href="')
                each_link_end = each_div_str.find("</a>")
                each_link = each_div_str[each_link_start + 9 : each_link_end]
                each_link_split = each_link.split('">')
                each_listing_title = (
                    each_link_split[1]
                    .replace("Promotion!", "")
                    .repalce("Sale!", "")
                    .strip()
                )
                each_listing_url = (
                    "https://XXXXXXXXXXXXXXXX.com/"
                    + each_link_split[0][each_link_split[0].find("products") :]
                )
                data[each_listing_title] = each_listing_url
    return data


def send_sms_alert(
    sms_url: str,
    username_token: str,
    password_token: str,
    to_phone_number: str,
    from_phone_number: str,
    body: str,
) -> requests.Response:
    """
    Sends SMS using Twilio REST API
    """
    response = requests.post(
        url=sms_url,
        auth=(username_token, password_token),
        data={
            "To": to_phone_number,
            "From": from_phone_number,
            "Body": body,
        },
    )
    return response


def run(
    sound_alert: bool = True, send_sms: bool = True, sleep_seconds: int = 350
) -> None:
    """
    Continuously checks website and alerts via SMS and audio when a change is detected
    """
    initial_time = datetime.datetime.now()
    old_listings = get_and_parse_urls(urls, headers)
    changes = []
    changes.append(f"{initial_time}: {len(old_listings)} INITIAL LISTINGS")
    while True:
        clear_output()
        try:
            # get content
            new_time = datetime.datetime.now()
            new_listings = get_and_parse_urls(urls, headers)
            # no changes detected
            if new_listings == old_listings:
                # history
                for each_change in changes:
                    print(each_change)
                # display listings
                print(f"{new_time}: NO CHANGES, {len(new_listings)} LISTINGS\n")
                print("CURRENT LISTINGS:")
                for each_key, each_value in new_listings.items():
                    print(f"{each_key}\n{each_value}")
            # changed detected
            else:
                # history
                for each_change in changes:
                    print(each_change)
                # changes
                print(f"{new_time}: WEBSITE UPDATED, {len(new_listings)} LISTINGS")
                new_keys = set(new_listings.keys())
                old_keys = set(old_listings.keys())
                added = new_keys - old_keys
                removed = old_keys - new_keys
                if added:
                    # display listings
                    print("\nNEW LISTINGS:")
                    for each in added:
                        print(f"{each}\n{new_listings[each]}")
                    # track changes
                    added_change = f"{new_time}: {len(added)} LISTINGS ADDED"
                    added_change += f" -> {', '.join(added)}"
                    changes.append(added_change)
                    # send sms alert
                    if send_sms:
                        message_body = f"{new_time} WEBSITE UPDATED!"
                        message_body += f" {len(added)} NEW LISTINGS!"
                        send_sms_alert(
                            sms_url,
                            username_token,
                            password_token,
                            to_phone_number,
                            from_phone_number,
                            message_body,
                        )
                    # computer alert
                    if sound_alert:
                        for _ in range(0, 10):
                            os.system('say "ALERT!"')
                if removed:
                    # display listings
                    print("\nREMOVED LISTINGS:")
                    for each in removed:
                        print(f"{each}\n{old_listings[each]}")
                    removed_change = f"{new_time}: {len(removed)} LISTINGS REMOVED"
                    removed_change += f" -> {', '.join(removed)}"
                    changes.append(removed_change)
                # update
                old_listings = new_listings
        except Exception as e:
            print(f"Exception: {e}")
            continue
        finally:
            time.sleep(sleep_seconds)

In [ ]:
# website urls to check for new product listings
urls = [
    "https://XXXXXXXXXXXXXXXX.com/collections/XXXXXXXXXXXXXXXX",
    "https://XXXXXXXXXXXXXXXX.com/collections/XXXXXXXXXXXXXXXX",
    "https://XXXXXXXXXXXXXXXX.com/collections/XXXXXXXXXXXXXXXX",
    "https://XXXXXXXXXXXXXXXX.com/collections/XXXXXXXXXXXXXXXX",
    "https://XXXXXXXXXXXXXXXX.com/collections/all?page=1",
    "https://XXXXXXXXXXXXXXXX.com/collections/all?page=2",
    "https://XXXXXXXXXXXXXXXX.com/collections/all?page=3",
    "https://XXXXXXXXXXXXXXXX.com/collections/all?page=4",
]

In [ ]:
# twilio REST API params for SMS message alert
sms_url = "https://api.twilio.com/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
username_token = "XXXXXXXXXXXXXXXX"
password_token = "XXXXXXXXXXXXXXXX"
to_phone_number = "+XXXXXXXXXXXXXXXX"
from_phone_number = "+XXXXXXXXXXXXXXXX"

In [ ]:
# runs main loop and disables computer sleep
try:
    disable_mac_sleep = subprocess.Popen(["caffeinate"])
    run()
finally:
    disable_mac_sleep.terminate()